# Set Up
Importing original data from https://www.aeaweb.org/articles?id=10.1257/aer.20131483

In [299]:
import pandas as pd
import numpy as np

xl = pd.ExcelFile("data.xlsx")
xl.sheet_names
df = xl.parse("Sheet1")

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14713 entries, 0 to 14712
Data columns (total 71 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   ID number                 14713 non-null  int64  
 1   Conveyance                14713 non-null  int64  
 2   Page                      14713 non-null  object 
 3   Researcher                14713 non-null  object 
 4   Notary First Name         14713 non-null  object 
 5   Notary Last Name          14713 non-null  object 
 6   Sales Date                14713 non-null  object 
 7   Sellers First Name        14713 non-null  object 
 8   Sellers Last Name         14701 non-null  object 
 9   Sellers County of Origin  14708 non-null  object 
 10  Sellers State of Origin   14711 non-null  object 
 11  Representing Seller       14577 non-null  object 
 12  Relationship to Seller    14574 non-null  object 
 13  Buyers First Name         14712 non-null  object 
 14  Buyers

# Cleaning up data
Dropping entries marked as omitted by the researchers and also the few entries missing vital information. I can't really figure out how the previous corgis set determined what entries to drop - it doesn't seem to be by the ommissions flag or by too much info missing. 

In [339]:
df = df[ df["DUMMY omission"] == 0 ]
df = df[df["Discount Rate"].notna()]
df = df[df["predicted rate"].notna()]
df = df[df["Price"].notna()]
df.drop_duplicates()
df.head()
#df = df[ df[""].notna()]

,ID number,Conveyance,Page,Researcher,Notary First Name,Notary Last Name,Sales Date,Sellers First Name,Sellers Last Name,Sellers County of Origin,...,Interest Rate,Discount Rate,predicted rate,Calculations,Ratio,PresentValue,DUMMY omission,Reason for Omission,Comments,DUMMY Estate Sale
0,1,70,348,MS,John French,Coffey,10/6/1856,Benjamin,Screws,.,...,NaN,0.006027,0.07477,NaN,NaN,NaN,0,NaN,NaN,NaN
1,2,70,349,MS,William,Shannon,10/7/1856,Santiago,Galseran,New Orleans,...,NaN,0.006027,0.07477,NaN,NaN,NaN,0,NaN,NaN,NaN
2,3,70,351,MS,Edward George,Gottschalk,10/1/1856,Pierre,Macou,New Orleans,...,NaN,0.006027,0.07477,NaN,NaN,NaN,0,NaN,NaN,NaN
3,4,70,352,MS,Paul Emile,Laresche,10/8/1856,Benjamin Franklin,Irby,Vanzand,...,NaN,0.006027,0.07477,NaN,NaN,NaN,0,NaN,NaN,NaN
4,5,70,352,MS,George,Rareshide,10/8/1856,Richard,Condon,New Orleans,...,NaN,0.006027,0.07477,NaN,NaN,NaN,0,NaN,NaN,NaN


In [346]:
#Copying over data to new dataframe
new_df = df[['Sales Date', 'Slave Name', 'Sex', 'Age', 'Color', 'Buyers County of Origin', 'Buyers State of Origin', 
            'Sellers County of Origin','Sellers State of Origin','Price', 'Payment Method', 
             'Number of Total Slaves','Number of Adult Slaves', 'Number of Child Slaves',
            'Discount Rate', 'predicted rate', 'Number of Prices']].copy()

#Name reformatting

#If they don't have a last name, usually a company name. Get rid of dot placeholder.
df.replace({'Sellers Last Name': {'.':''}},inplace=True)
df.replace({'Buyers Last Name': {'.':''}},inplace=True)

#Concatenate first and last names
new_df['Buyers Full Name'] = df['Buyers First Name'] + ' ' + df['Buyers Last Name']
new_df['Sellers Full Name'] = df['Sellers First Name'] + ' ' + df['Sellers Last Name']


#Change time to datetime format for easy sorting, separate times
dti = pd.DatetimeIndex(df['Sales Date'])
new_df['Sales Date'] = dti
new_df['Sales Year'] = dti.year 
new_df['Sales Month'] = dti.month
new_df['Sales Day'] = dti.day

#Replace all dot placeholders and NaNs with 'Unknown'
new_df.replace('.','Unknown',inplace=True)
new_df.replace(np.nan ,'Unknown',inplace=True )

#Fully written out gender
new_df.replace({'Sex': {'M' :'Male', 'F' : 'Female'} },inplace=True )

#Rounding since some of these floats are really long
new_df = new_df.round({'Discount Rate': 8, 'predicted rate': 8})

new_df.head()

,Sales Date,Slave Name,Sex,Age,Color,Buyers County of Origin,Buyers State of Origin,Sellers County of Origin,Sellers State of Origin,Price,...,Number of Adult Slaves,Number of Child Slaves,Discount Rate,predicted rate,Number of Prices,Buyers Full Name,Sellers Full Name,Sales Year,Sales Month,Sales Day
0,1856-10-06,Shadrack,Male,40,Unknown,Unknown,Unknown,Unknown,AL,650.0,...,1,0,0.006027,0.07477,1,Maria Hopson,Benjamin Screws,1856,10,6
1,1856-10-07,Jane,Female,44,Negro,New Orleans,LA,New Orleans,LA,550.0,...,1,0,0.006027,0.07477,1,Peyton N. Reynolds,Santiago Galseran,1856,10,7
2,1856-10-01,Helen,Female,24,Negro,New Orleans,LA,New Orleans,LA,1000.0,...,1,0,0.006027,0.07477,1,Heloise Bagneris,Pierre Macou,1856,10,1
3,1856-10-08,Jeff,Male,23,Negro,Jefferson,LA,Vanzand,TX,1400.0,...,1,0,0.006027,0.07477,1,Francis Dugue Jr.,Benjamin Franklin Irby,1856,10,8
4,1856-10-08,John,Male,12,Negro,New Orleans,LA,New Orleans,LA,700.0,...,1,0,0.006027,0.07477,1,Anthony O'Dowd,Richard Condon,1856,10,8


# Re-ordering
Sorting rows by date and also column names 

In [347]:
#Column name order
columnsTitles = ['Sales Date',
                 'Sales Year',
                 'Sales Month',
                 'Sales Day',
                 'Slave Name', 
                 'Sex', 
                 'Age', 
                 'Color', 
                 'Sellers Full Name',
                 'Sellers County of Origin',
                 'Sellers State of Origin', 
                 'Buyers Full Name',
                 'Buyers County of Origin', 
                 'Buyers State of Origin', 
                 'Price', 
                 'Payment Method', 
                 'Number of Total Slaves',
                 'Number of Adult Slaves',
                 'Number of Child Slaves',
                 'Number of Prices',
                 'Discount Rate',
                 'predicted rate', ]

new_df = new_df.reindex(columns=columnsTitles)

#Set index to sales date and reorder by date
new_df.set_index("Sales Date", drop=True, append=False, inplace=True)
new_df = new_df.sort_values(by="Sales Date")

new_df.head(10)

,Sales Year,Sales Month,Sales Day,Slave Name,Sex,Age,Color,Sellers Full Name,Sellers County of Origin,Sellers State of Origin,...,Buyers County of Origin,Buyers State of Origin,Price,Payment Method,Number of Total Slaves,Number of Adult Slaves,Number of Child Slaves,Number of Prices,Discount Rate,predicted rate
Sales Date,,,,,,,,,,,,,,,,,,,,,
1856-05-03,1856,5,3,Rose,Female,21,Negro,"Josephe Marcel, Anais Gordain Braud",St. James,LA,...,New Orleans,LA,850.0,Cash,1,1,0,1,0.006027,0.074770
1856-10-01,1856,10,1,Eliza,Female,30,Unknown,John S. Maunsell,New Orleans,LA,...,New Orleans,LA,925.0,Cash and Credit,1,1,0,1,0.005791,0.071746
1856-10-01,1856,10,1,Helen,Female,24,Negro,Pierre Macou,New Orleans,LA,...,New Orleans,LA,1000.0,Cash,1,1,0,1,0.006027,0.074770
1856-10-01,1856,10,1,Green,Male,30,Mulatto,John Caldwell Jr.,New Orleans,LA,...,New Orleans,LA,900.0,Cash,1,1,0,1,0.006027,0.074770
1856-10-01,1856,10,1,Catherine,Female,32,Creole Mulatto,Marie Jeanne Victoire Dubertrand,New Orleans,LA,...,New Orleans,LA,875.0,Cash,1,1,0,1,0.006027,0.074770
1856-10-01,1856,10,1,Jane,Female,13,Negro,William Jackson Maynard,New Orleans,LA,...,New Orleans,LA,812.0,Cash,1,1,0,1,0.006027,0.074770
1856-10-02,1856,10,2,Mary/ Louise,Female,35,Creole Negro,Claude Dolbeau,New Orleans,LA,...,New Orleans,LA,500.0,Cash and Credit,1,1,0,1,0.005791,0.071746
1856-10-02,1856,10,2,Rosalie,Female,37,Negro,Felicie Dumaine,New Orleans,LA,...,New Orleans,LA,700.0,Cash and Credit,1,1,0,1,0.005791,0.071746
1856-10-04,1856,10,4,Hayes,Male,35,Negro,Mary Doyle,New Orleans,LA,...,New Orleans,LA,750.0,Cash,1,1,0,1,0.006027,0.074770


# Exporting
Export to csv without column names

In [348]:
new_df.to_csv('slavery-corgis.csv', sep=',' ,header=False)
new_df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 10514 entries, 1856-05-03 to 1861-09-03
Data columns (total 21 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Sales Year                10514 non-null  int64  
 1   Sales Month               10514 non-null  int64  
 2   Sales Day                 10514 non-null  int64  
 3   Slave Name                10514 non-null  object 
 4   Sex                       10514 non-null  object 
 5   Age                       10514 non-null  object 
 6   Color                     10514 non-null  object 
 7   Sellers Full Name         10514 non-null  object 
 8   Sellers County of Origin  10514 non-null  object 
 9   Sellers State of Origin   10514 non-null  object 
 10  Buyers Full Name          10514 non-null  object 
 11  Buyers County of Origin   10514 non-null  object 
 12  Buyers State of Origin    10514 non-null  object 
 13  Price                     10514 non-null  fl